In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy import ndimage

In [3]:
rawImage = ndimage.imread('./lighthouse_RAW_noisy_sigma0.01.png')
import numpy as np

width, height = rawImage.shape
pattern = [['r', 'g'],
           ['g', 'b']]

def linearInterpolate(rawImage, targetColor, pattern, x, y):
    patternDelta = (x % 2, y % 2)
    pixelColorType = pattern[patternDelta[0]][patternDelta[1]]
    
    targetColorDelta = (-1, -1)
    if targetColor in pattern[0]:
        targetColorDelta = (pattern[0].index(targetColor), 0)
    if targetColor in pattern[1]:
        targetColorDelta = (pattern[1].index(targetColor), 1)

    if targetColorDelta == (-1, -1):
        return 0

    xOffset = abs(patternDelta[0] - targetColorDelta[0])
    yOffset = abs(patternDelta[1] - targetColorDelta[1])

    interpolationPoints = [
        (x - xOffset, y - yOffset),
        (x + xOffset, y - yOffset),
        (x - xOffset, y + yOffset),
        (x + xOffset, y + yOffset)
    ]
    
    interpolationPoints = filter(lambda xy: xy[0] > -1 and xy[0] < rawImage.shape[0], interpolationPoints)
    interpolationPoints = filter(lambda xy: xy[1] > -1 and xy[1] < rawImage.shape[1], interpolationPoints)
    interpolationPoints = list(set(interpolationPoints))
    valuePoints = list(map(lambda xy: rawImage[xy], interpolationPoints))
    return float(sum(valuePoints)) / len(valuePoints)
        
def computeColor(rawImage, pattern, targetColor, isReal, x, y):
    if isReal:
        return rawImage[x, y]
    else:        
        return linearInterpolate(rawImage, targetColor, pattern, x, y)
    
result = []
for x in range(width):
    for y in range(height):
        isRealR = pattern[x % 2][y % 2] == 'r'
        isRealG = pattern[x % 2][y % 2] == 'g'
        isRealB = pattern[x % 2][y % 2] == 'b'
        
        rValue = computeColor(rawImage, pattern, 'r', isRealR, x, y)
        gValue = computeColor(rawImage, pattern, 'g', isRealG, x, y)
        bValue = computeColor(rawImage, pattern, 'b', isRealB, x, y)
        result.append((rValue, gValue, bValue))
        
result = np.asarray(result).reshape(width, height, 3)
imshow(result / 255.0)